In [1]:
# Libraries
import itertools
from importlib import resources
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import time

from drdt.helper_functions import DecisionRuleCreatorFromDecisionTable, Reduction, R_SR, R_AD, SAlphaStep, SPlus, SMax, NCover, NGreedy
from drdt.algorithms import DynamicProgrammingAlgorithms, A_C_N, A_C_G


# Loading Data

In [2]:
with resources.path('datasets.DecisionRuleSystems', 'DRS_breast_cancer') as dataset_path:
    S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)
S = S.dropna(axis=1, how='all') # Drop the columns with all None
S

/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_7143/4260445852.py:1: DeprecationWarning: path is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  with resources.path('datasets.DecisionRuleSystems', 'DRS_breast_cancer') as dataset_path:
/var/folders/06/0m2hb6f927s6rdyhxjrbss740r2h_1/T/ipykernel_7143/4260445852.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  S = pd.read_csv(dataset_path).applymap(lambda x: str(x) if pd.notnull(x) else x)


,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,class
0,20-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
1,30-39,lt40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
2,NaN,NaN,0-4,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
3,30-39,NaN,14-Oct,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
4,30-39,NaN,14-Oct,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
...,...,...,...,...,...,...,...,...,...,...
261,70-79,NaN,14-Oct,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
262,70-79,NaN,NaN,11-Sep,NaN,NaN,NaN,NaN,NaN,recurrence-events
263,70-79,NaN,20-24,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events
264,70-79,NaN,40-44,NaN,NaN,NaN,NaN,NaN,NaN,no-recurrence-events


# Dataset Analyses

In [3]:
n = len(S.columns)-1
print(f"Number of features  = {n}")

Number of features  = 9


In [4]:
print(f"Does # of features = d  = {not S.dropna().empty}")

Does # of features = d  = False


In [5]:
# Count non-NaN values for each row
non_nan_counts = S.count(axis=1)

# Find the index
max_non_nan_row_index = non_nan_counts.idxmax()

# Retrieve the row
max_non_nan_row = S.loc[max_non_nan_row_index]

# Number of non-NaN values in the row
max_non_nan_count = non_nan_counts[max_non_nan_row_index]

d = max_non_nan_count - 1

print(f"d = {d}") # remove 1 because of last class column

d = 6


In [6]:
P = S
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AR = {len(B)}")

Length of Node Cover for AR = 6


In [7]:
P = R_SR(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for SR = {len(B)}")

Length of Node Cover for SR = 6


In [8]:
P = R_AD(S)
P_plus = SPlus(P)
B = NCover(P_plus)
print(f"Length of Node Cover for AD = {len(B)}")

Length of Node Cover for AD = 6


In [9]:
P = S
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AR = {len(B)}")

Length of Node Cover greedy for AR = 1


In [10]:
P = R_SR(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for SR = {len(B)}")

Length of Node Cover greedy for SR = 1


In [11]:
P = R_AD(S)
P_plus = SPlus(P)
B = NGreedy(P_plus)
print(f"Length of Node Cover greedy for AD = {len(B)}")

Length of Node Cover greedy for AD = 1


# Combinations

In [12]:
column_values = []

for column in S.columns[:-1]: #ignoring class column
    column_values.append(list(S[column].dropna().unique()))
    
# All possible combinations    
combinations = list(itertools.product(*column_values))

print('Number of Possible combinations =', len(combinations))

Number of Possible combinations = 166320


In [13]:
ext_column_values = []

for column in S.columns[:-1]: #ignoring class column
    ext_column_values.append(list(S[column].dropna().unique()) + ['*'])
    
# All possible combinations with extension    
ext_combinations = list(itertools.product(*ext_column_values))

print('Number of Possible ext_combinations =', len(ext_combinations))

Number of Possible ext_combinations = 1741824


# Node Cover

### AR

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

### EAR

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

### SR

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

### ESR

In [24]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1741824/1741824 [8:07:54<00:00, 59.50it/s]

NC ESR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 6, 6.4489414544753085, 29274.207304239273)


### AD

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

### EAD

In [23]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="cover")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NC EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1741824/1741824 [8:57:13<00:00, 54.04it/s]

NC EAD
(Max Depth, Min Depth, Average Depth, Running time) = (9, 6, 6.813469673170194, 32233.652045965195)


# Node Cover Greedy

### AR

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

### EAR

In [22]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1741824/1741824 [4:05:02<00:00, 118.47it/s]

NCgreedy EAR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 4, 6.463882688492063, 14702.670506238937)


### SR

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="SR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

### ESR

In [21]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="ESR", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1741824/1741824 [10:39:10<00:00, 45.42it/s]

NCgreedy ESR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 1, 4.381360573743386, 38350.63984799385)


### AD

In [ ]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="AD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

### EAD

In [20]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_N(C="EAD", N="greedy")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("NCgreedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1741824/1741824 [11:58:26<00:00, 40.41it/s]

NCgreedy EAD
(Max Depth, Min Depth, Average Depth, Running time) = (9, 3, 5.5219287367724865, 43106.60940718651)


# Greedy

### AR

In [14]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy AR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 166320/166320 [08:38<00:00, 320.75it/s]

Greedy AR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 4, 6.488552188552188, 518.5946550369263)


### EAR

In [17]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy EAR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1741824/1741824 [1:27:41<00:00, 331.06it/s]

Greedy EAR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 4, 6.20148189484127, 5261.428818941116)


### SR

In [15]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="SR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy SR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 166320/166320 [49:17<00:00, 56.24it/s]

Greedy SR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 2, 3.9224987974987977, 2957.263598203659)


### ESR

In [18]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="ESR")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy ESR")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1741824/1741824 [8:36:52<00:00, 56.17it/s]

Greedy ESR
(Max Depth, Min Depth, Average Depth, Running time) = (9, 2, 4.151437803130511, 31012.487036943436)


### AD

In [16]:
start_time = time.time()

depths = []

for comb in tqdm(combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="AD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy AD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 166320/166320 [54:27<00:00, 50.91it/s]

Greedy AD
(Max Depth, Min Depth, Average Depth, Running time) = (9, 3, 5.105651755651755, 3267.2026896476746)


### EAD

In [19]:
start_time = time.time()

depths = []

for comb in tqdm(ext_combinations):
    #creating delta
    delta = pd.DataFrame([list(comb)], columns=list(S.columns[:-1]))
    
    delta = delta.loc[0]
    
    alg = A_C_G(C="EAD")
    
    depth, _ = alg.solve(S, delta=delta)
    
    depths.append(depth)

end_time = time.time()
    
print("Greedy EAD")    
print("(Max Depth, Min Depth, Average Depth, Running time) =", (max(depths), min(depths), sum(depths)/len(depths), end_time - start_time))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1741824/1741824 [9:25:28<00:00, 51.34it/s]

Greedy EAD
(Max Depth, Min Depth, Average Depth, Running time) = (9, 3, 5.069160259589947, 33928.60415101051)
